In [2]:
## License: Apache 2.0. See LICENSE file in root directory.
## Copyright(c) 2017 Intel Corporation. All Rights Reserved.

#####################################################
##              Align Depth to Color               ##
#####################################################

# First import the library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2

from skimage import io, morphology


# Create a pipeline
pipeline = rs.pipeline()

#Create a config and configure the pipeline to stream
#  different resolutions of color and depth streams
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
profile = pipeline.start(config)

# Getting the depth sensor's depth scale (see rs-align example for explanation)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()
print("Depth Scale is: " , depth_scale)

# We will be removing the background of objects more than
#  clipping_distance_in_meters meters away
clipping_distance_in_meters = .4 #1 meter
clipping_distance = clipping_distance_in_meters / depth_scale

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)
img_counter = 13
#square = morphology.square(3)
slem =  morphology.disk(4)

# Streaming loop
try:
    while True:
        # Get frameset of color and depth
        frames = pipeline.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() # aligned_depth_frame is a 640x480 depth image
        #color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame:
            continue

        depth_image = np.asanyarray(aligned_depth_frame.get_data())
        #color_image = np.asanyarray(color_frame.get_data())

        # Remove background - Set pixels further than clipping_distance to grey
        grey_color = 0
        #depth_image_3d = np.dstack((depth_image,depth_image,depth_image)) #depth image is 1 channel, color is 3 channels
        #bg_removed = np.where((depth_image_3d > clipping_distance)| (depth_image_3d <= 0), grey_color, color_image)
        depth_corrected = np.where((depth_image > clipping_distance)| (depth_image <= 0), 0, 1.0)
        
        after = morphology.closing(depth_corrected,selem=slem)
        #skeletonized = skeletonize(depth_corrected/depth_corrected.max())
        # Render images
        depth_map = cv2.convertScaleAbs(depth_image, alpha=0.03)
        cv2.imshow("b4", depth_corrected)
        
        cv2.imshow("after", after)
        key = cv2.waitKey(1)
        
        if(key & 0xFF == ord('s')):
            cv2.imwrite('img_papel'+str(img_counter)+".png", after)
            img_counter+= 1
        
        # Press esc or 'q' to close the image window
        if key & 0xFF == ord('q') or key == 27:
            cv2.destroyAllWindows()
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()

Depth Scale is:  0.0010000000474974513


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,16))
plt.imshow(depth_corrected)
plt.show()
plt.figure(figsize=(16,16))
plt.imshow(skeletonized)
plt.show()